In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# connect TFRecords to graph

In [2]:
data_path = '/shared/TORCS_train.tfrecords'

feature = {'fast': tf.FixedLenFeature([], tf.float32),
    'dist_RR': tf.FixedLenFeature([], tf.float32),
    'dist_MM': tf.FixedLenFeature([], tf.float32),
    'dist_LL': tf.FixedLenFeature([], tf.float32),
    'toMarking_RR': tf.FixedLenFeature([], tf.float32),
    'toMarking_MR': tf.FixedLenFeature([], tf.float32),
    'toMarking_ML': tf.FixedLenFeature([], tf.float32),
    'toMarking_LL': tf.FixedLenFeature([], tf.float32),
    'dist_R': tf.FixedLenFeature([], tf.float32),
    'dist_L': tf.FixedLenFeature([], tf.float32),
    'toMarking_R': tf.FixedLenFeature([], tf.float32),
    'toMarking_M': tf.FixedLenFeature([], tf.float32),
    'toMarking_L': tf.FixedLenFeature([], tf.float32),
    'angle': tf.FixedLenFeature([], tf.float32),
    'image': tf.FixedLenFeature([], tf.string)}

# Create a list of filenames and pass it to a queue
filename_queue = tf.train.string_input_producer([data_path], num_epochs=100)
# Define a reader and read the next record
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
# Decode the record read by the reader
features = tf.parse_single_example(serialized_example, features=feature)
# Convert the image data from string back to the numbers
image = tf.decode_raw(features['image'], tf.uint8)

# Cast label data into float32
fast = tf.cast(features['fast'], tf.float32)

# rescale all targets to [0,1], numbers were taken from source code of DeepDriving
dist_RR = tf.cast(features['dist_RR'], tf.float32) / 95.0 + 0.12
dist_MM = tf.cast(features['dist_MM'], tf.float32) / 95.0 + 0.12
dist_LL = tf.cast(features['dist_LL'], tf.float32) / 95.0 + 0.12

toMarking_RR = tf.cast(features['toMarking_RR'], tf.float32) / 6.8752 - 0.48181
toMarking_MR = tf.cast(features['toMarking_MR'], tf.float32) / 6.25 + 0.02
toMarking_ML = tf.cast(features['toMarking_ML'], tf.float32) / 6.25 + 0.98
toMarking_LL = tf.cast(features['toMarking_LL'], tf.float32) / 6.8752 + 1.48181

dist_R = tf.cast(features['dist_R'], tf.float32) / 95.0 + 0.12
dist_L = tf.cast(features['dist_L'], tf.float32) / 95.0 + 0.12

toMarking_R = tf.cast(features['toMarking_R'], tf.float32) / 5.6249 - 0.34445
toMarking_M = tf.cast(features['toMarking_M'], tf.float32) / 6.8752 + 0.39091
toMarking_L = tf.cast(features['toMarking_L'], tf.float32) / 5.6249 + 1.34445

angle = tf.cast(features['angle'], tf.float32) / 1.1 + 0.5

# Reshape image data into the original shape
image = tf.reshape(image, [210, 280, 3])

# rescale from [0..255] to [-1..1]
image = tf.cast(image, tf.float32)
image = (image - 128.0) / 128.0

# Creates batches by randomly shuffling tensors
#images=1
images, fasts, dist_RRs, dist_MMs, dist_LLs, toMarking_RRs, toMarking_MRs, toMarking_MLs, toMarking_LLs, dist_Rs, dist_Ls, toMarking_Rs, toMarking_Ms, toMarking_Ls, angles = tf.train.shuffle_batch(
                    [image, fast, dist_RR, dist_MM, dist_LL, toMarking_RR, toMarking_MR, toMarking_ML,
                        toMarking_LL, dist_R, dist_L, toMarking_R, toMarking_M, toMarking_L, angle],
                        batch_size=64, capacity=50000, num_threads=8, min_after_dequeue=10000, allow_smaller_final_batch=True)


In [3]:
y_true = tf.concat([tf.expand_dims(fasts,1), tf.expand_dims(dist_RRs,1), tf.expand_dims(dist_MMs,1), tf.expand_dims(dist_LLs,1), 
                    tf.expand_dims(toMarking_RRs,1), tf.expand_dims(toMarking_MRs,1), tf.expand_dims(toMarking_MLs,1),
                    tf.expand_dims(toMarking_LLs,1), tf.expand_dims(dist_Rs,1), tf.expand_dims(dist_Ls,1), tf.expand_dims(toMarking_Rs,1),
                    tf.expand_dims(toMarking_Ms,1), tf.expand_dims(toMarking_Ls,1), tf.expand_dims(angles,1)], axis=1)

# Alexnet

In [4]:
from alexnet import alexnet_v2


logits, endpoints = alexnet_v2(images,
               num_classes=1000,
               is_training=True,
               dropout_keep_prob=0.999,
               spatial_squeeze=True,
               scope='alexnet_v2')

y_pred = endpoints['alexnet_v2/fc8']
loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))

### input and output Tensor names

In [ ]:
images

In [10]:
endpoints

OrderedDict([('alexnet_v2/conv1',
              <tf.Tensor 'alexnet_v2/conv1/Relu:0' shape=(64, 50, 68, 64) dtype=float32>),
             ('alexnet_v2/pool1',
              <tf.Tensor 'alexnet_v2/pool1/MaxPool:0' shape=(64, 24, 33, 64) dtype=float32>),
             ('alexnet_v2/conv2',
              <tf.Tensor 'alexnet_v2/conv2/Relu:0' shape=(64, 24, 33, 192) dtype=float32>),
             ('alexnet_v2/pool2',
              <tf.Tensor 'alexnet_v2/pool2/MaxPool:0' shape=(64, 11, 16, 192) dtype=float32>),
             ('alexnet_v2/conv3',
              <tf.Tensor 'alexnet_v2/conv3/Relu:0' shape=(64, 11, 16, 384) dtype=float32>),
             ('alexnet_v2/conv4',
              <tf.Tensor 'alexnet_v2/conv4/Relu:0' shape=(64, 11, 16, 384) dtype=float32>),
             ('alexnet_v2/conv5',
              <tf.Tensor 'alexnet_v2/conv5/Relu:0' shape=(64, 11, 16, 256) dtype=float32>),
             ('alexnet_v2/pool5',
              <tf.Tensor 'alexnet_v2/pool5/MaxPool:0' shape=(64, 5, 7, 256) dtyp

# Mobilenet

In [4]:
# from mobilenet_v1 import mobilenet_v1

# logits, endpoints = mobilenet_v1(images, #tf.cast(images, tf.float32),
#                  num_classes=1000,
#                  dropout_keep_prob=0.999,
#                  is_training=True,
#                  min_depth=8,
#                  depth_multiplier=0.75,
#                  conv_defs=None,
#                  prediction_fn=tf.contrib.layers.softmax,
#                  spatial_squeeze=True,
#                  reuse=None,
#                  scope='MobilenetV1')

# y_pred = endpoints['Logits']
# loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))

# Training

In [ ]:
from time import time

start_t = time()

train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

saver = tf.train.Saver(max_to_keep=None)

with tf.Session() as sess:

    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)

    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    loss_list = []
    for batch_index in range(140000):
        loss_val, _ = sess.run([loss, train_step])
        loss_list.append((loss_val, time() - start_t))
        if batch_index % 10 == 0:
            np.save('loss.npy', loss_list)
        if (batch_index+1) % 20000 == 0:
            saver.save(sess, "model_alex/{}.ckpt".format(batch_index))
    
    # Stop the threads
    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)
    sess.close()

INFO:tensorflow:model_alex/19999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/39999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/59999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/79999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/99999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/119999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/139999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
